# Estudo de Caso - Limpeza e Pré-Processamento de Dados com NumPy

### Contexto:

Imagine que em um determinado projeto de Ciência de Dados você receba um dataset extremamente complicado, contendo dados com muitas strings, caracteres especiais, problemas de encoding, datas  mal  formatadas,  números  e  textos  na  mesma  coluna,  url’s  contendo  Ids importantes para análise, valores ausentes, coluna que contém informação que deveria estar distribuída  em três ou  mais  colunas. E  como  se  não  bastasse  tudo  isso,  parte  dos  dados necessários para análise está em outro dataset, que deve ser combinado com o primeiro.

Seu trabalho seria limpar e pré-processar esse dataset, preparando-o para a sequência do processo de análise.

É exatamente este cenário que estamos reproduzindo no Estudo de Caso 1. A partir de dados complexos e com diversos problemas, iremos fazer um extenso trabalho de limpeza e pré-processamento. E tudo isso usando apenas o NumPy,  poderoso pacote da Linguagem Python para computação e processamento de dados.

#### Este Estudo de Caso traz uma quantidade incrível de conhecimento sobre manipulação de dados em Python.

## Definição do Problema e Fonte de Dados

#### Para este Estudo de Caso trabalharemos com dado reais disponíveis publicamente no link abaixo:
    
    https://www.openintro.org/data/index.php?data=loans_full_schema

Esse conjunto de dados representa milhares de empréstimos feitos por meio da plataforma Lending Club, que é uma
plataforma que permite que indivíduos emprestem para outros indivíduos.

Claro, nem todos os empréstimos são iguais. Alguém que fornece um baixo risco e que provavelmente vai pagar um
empréstimo terá mais facilidade em obter um empréstimo com uma taxa de juros baixa do que alguém que parece ser
mais arriscado.

E para as pessoas com alto risco de não pagar o empréstimo? Essas pessoas podem nem receber uma oferta de empréstimo, ou podem não aceitar uma oferta de empréstimo devido a uma alta taxa de juros. É importante ter em mente essa última parte, pois esse conjunto de dados representa  apenas  empréstimos  efetivamente  feitos,  ou  seja,  não  confunda  esses  dados  com pedidos de empréstimo!

Usamos como fonte de dados o dataset disponível no link acima, mas fizemos modificações nos dados para deixá-los ainda mais problemáticos. O dataset será fornecido a você junto com os demais arquivos do capítulo.

Além  disso  usaremos  um  dataset  com  cotação  do  dólar  em  relação  ao  Euro.  Extraímos uma pequena amostra de dados do site: https://finance.yahoo.com. O dataset será fornecido a você junto com os demais arquivos do capítulo.

## Objetivo:

Nosso trabalho é limpar e pré-processar os dados, deixando-os no formato ideal para um processo de análise posterior e várias decisões terão que ser tomadas no meio do caminho. Ao final do trabalho devemos salvar o dataset com os dados limpos e pré-processados

## Importando Pacotes e Configurando Impressão do NumPy

In [23]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress = True, linewidth = 200, precision = 2)

## Verificando a Codificação do Arquivo

In [24]:
import chardet

with open('datasets/dataset1.csv', 'rb') as f:
    result = chardet.detect(f.read())  # Lê uma amostra do arquivo
    print(result['encoding'])

windows-1251


## Carregando e Visualizando o Dataset

In [32]:
dados = np.genfromtxt("datasets/dataset1.csv",
                     delimiter = ';',
                     skip_header = 1,              # não carrega cabeçalho 
                     autostrip = True,
                     encoding = 'windows-1251'     # 'cp1252'
                     )
dados


array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

### Utilizando Pandas Apenas Para Visualizar o Dataset

In [29]:
import pandas as pd

# Convertendo o array NumPy para um DataFrame do Pandas
df = pd.DataFrame(dados)

# Exibindo o DataFrame
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,48010226.0,NaN,35000.0,NaN,35000.0,NaN,13.33,1184.86,NaN,NaN,NaN,NaN,NaN,9452.96
1,57693261.0,NaN,30000.0,NaN,30000.0,NaN,NaN,938.57,NaN,NaN,NaN,NaN,NaN,4679.70
2,59432726.0,NaN,15000.0,NaN,15000.0,NaN,NaN,494.86,NaN,NaN,NaN,NaN,NaN,1969.83
3,53222800.0,NaN,9600.0,NaN,9600.0,NaN,NaN,300.35,NaN,NaN,NaN,NaN,NaN,1793.68
4,57803010.0,NaN,8075.0,NaN,8075.0,NaN,19.19,296.78,NaN,NaN,NaN,NaN,NaN,1178.51
5,63398019.0,NaN,14400.0,NaN,14400.0,NaN,13.99,334.99,NaN,NaN,NaN,NaN,NaN,681.17
6,60850626.0,NaN,NaN,NaN,13000.0,NaN,24.99,381.50,NaN,NaN,NaN,NaN,NaN,1126.45
7,46816139.0,NaN,7500.0,NaN,7500.0,NaN,13.33,253.90,NaN,NaN,NaN,NaN,NaN,2025.65
8,63918356.0,NaN,8000.0,NaN,8000.0,NaN,NaN,250.29,NaN,NaN,NaN,NaN,NaN,497.07
9,63651616.0,NaN,7200.0,NaN,7200.0,NaN,NaN,241.18,NaN,NaN,NaN,NaN,NaN,512.57


Observe como várias colunas estão com o tipo NaN. Isso se deve a caracteres especiais no conjunto de dados e a forma como o NumPy carrega dados numéricos e do tipo string. Iremos resolver isso

### Tipo de Dados

In [35]:
print(type(dados))
print(type(df))

print(dados.shape)
print(df.shape)

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
(10000, 14)
(10000, 14)
